First let's take a look at our model to dataset(s) linking use case -

In [46]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain.schema import Document
import os
from llm_tools import process_ask_gpt_in_parallel,parse_json_response,ask_gpt,ask_gpt_with_continue
import glob
import json
from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)
import os
import time
from extract.CodeLATS.code_lats import use_lats
from extract.main import *

In [2]:
docs_directory='/media/hdd/Code/auto-config/WRF/external_docs';code_directory='/media/hdd/Code/auto-config/WRF/test';library_name='WRF'

In [3]:
doc_files=glob.glob(docs_directory+'/**',recursive=True)
docs_in_string_format=[]
valid_doc_files=[]
for file_path in doc_files:
    if '.txt' in file_path:
        with open(file_path, 'r') as file:
            file_content = file.read()
        docs_in_string_format.append(file_content)
        valid_doc_files.append(file_path)
print(len(valid_doc_files))

453


In [4]:
#############   STEP 1: get summaries of documentation articles ####################
prompts={j: SUMMARIZE_DOCUMENTATION_DOC_PROMPT_TEMPLATE.format(library_name=library_name,article=doc) 
         for j,doc in enumerate(docs_in_string_format)}
summarize_docs_time=time.time()
responses=process_ask_gpt_in_parallel(prompts.values(), prompts.keys(), model='gpt-3.5-turbo-1106',max_workers=24,response_format={"type": "json_object"}) 
print('Took this much time to summarize the docs - ',time.time()-summarize_docs_time)
sorted_keys = sorted(responses.keys())
responses = {key: responses[key] for key in sorted_keys}
all_summaries = ' '.join([f"Article {i} :\n Title: {valid_doc_files[i]} \n Summary: {json.loads(responses[key])['summary']}\n\n" for i,key in enumerate(responses.keys())])
print(responses[0],'\n',responses[1],'\n',responses[2])

Processing Prompts:  38%|███▊      | 173/453 [00:36<00:34,  8.21it/s]

Trimming prompt from 10789 characters to fit within token limit.


Processing Prompts:  45%|████▌     | 205/453 [00:42<00:47,  5.19it/s]

Trimming prompt from 4301 characters to fit within token limit.


Processing Prompts:  49%|████▉     | 222/453 [00:44<00:33,  6.83it/s]

Trimming prompt from 4484 characters to fit within token limit.


Processing Prompts:  64%|██████▎   | 288/453 [00:54<00:12, 12.82it/s]

Trimming prompt from 5293 characters to fit within token limit.


Processing Prompts: 100%|██████████| 453/453 [01:24<00:00,  5.38it/s]

Took this much time to summarize the docs -  84.24326491355896
{
  "summary": "The WRF code's build mechanism determines the computing system's architecture and presents build options. It prompts for the desired usage of processors and only allows selecting a compiler installed on the system. The How to Compile WRF website provides steps for building the WRF and WPS codes, and obtaining the latest version of the code is recommended. Versions prior to V4.0 are no longer supported."
} 
 {
  "summary": "The WRF Preprocessing System (WPS) requires WRF to be compiled prior to its installation so that the I/O API libraries in the WRF external directory will be available to WPS programs. The only library required to build the WRF model and WPS is netCDF, with ungrib program needing three compression libraries for GRIB Edition 2 support. NCEP GRIB libraries, w3 and g2, are included in the WPS distribution to address portability issues. The How to Compile WRF website provides guidance for build

In [5]:
#############   STEP 2: WHICH DOCS TO READ  ####################
which_articles_prompt=WHICH_DOC_TO_READ_RUN_PROMPT_TEMPLATE.format(library_name=library_name,summaries=all_summaries,n=20,specific_request='Specifically you are trying to run idealized simulations')#'Specifically you are trying to run idealized simulations')
choose_docs_time=time.time()
out = ask_gpt(which_articles_prompt, model='gpt-4-1106-preview',response_format={"type": "json_object"})
print('Took this much time to choose which docs to read - ',time.time()-choose_docs_time)
run_articles_to_read=json.loads(out)['articles']
print(run_articles_to_read)

Took this much time to choose which docs to read -  22.311413764953613
[12, 13, 14, 15, 17, 18, 19, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 52]


In [6]:
#############   STEP 3: GET CONFIG PARAMS FROM DOCS/CODE  ####################
   ########   STEP 3a: GET CONFIG FILES FROM DOCS  #############

example_run_doc_1=docs_in_string_format[37] 
part12_prompt=HOW_TO_RUN_PROMPT_TEMPLATE_part_12.format(library_name=library_name,article=example_run_doc_1)
read_doc_time=time.time()
out = ask_gpt(part12_prompt, model='gpt-4-1106-preview',response_format={"type": "json_object"})
print('Took this much time to get config files from docs - ',time.time()-read_doc_time)
part12_extract=json.loads(out)   
modification_code_files=part12_extract['modification_code_files']
print(modification_code_files)

Took this much time to get config files from docs -  55.47008156776428
['WRF/test/em_b_wave/namelist.input', 'WRF/run/README.namelist']


In [48]:
########   STEP 3b: GET CONFIG PARAMS FROM CODE  #############
extracted_file_names=['WRF/test/em_b_wave/namelist.input', 'WRF/run/README.namelist']
with open(os.path.join(code_directory,'em_b_wave/namelist.input'),'r') as f:
    example_run_code_1_file_1=f.read()

In [38]:
read_code_time=time.time()
conf_result_1=ask_gpt(get_config_simple_1.format(config_text=example_run_code_1_file_1),'gpt-3.5-turbo-1106',response_format={"type": "json_object"})
print('Took this much time to get config params from code - ',time.time()-read_code_time)
variables=json.loads(conf_result_1)['configuration_variables']
print(len(variables))
print(variables)

Took this much time to get config params from code -  6.85133695602417
86
['run_days', 'run_hours', 'run_minutes', 'run_seconds', 'start_year', 'start_month', 'start_day', 'start_hour', 'start_minute', 'start_second', 'end_year', 'end_month', 'end_day', 'end_hour', 'end_minute', 'end_second', 'history_interval', 'frames_per_outfile', 'restart', 'restart_interval', 'io_form_history', 'io_form_restart', 'io_form_input', 'io_form_boundary', 'time_step', 'time_step_fract_num', 'time_step_fract_den', 'max_dom', 's_we', 'e_we', 's_sn', 'e_sn', 's_vert', 'e_vert', 'dx', 'dy', 'ztop', 'grid_id', 'parent_id', 'i_parent_start', 'j_parent_start', 'parent_grid_ratio', 'parent_time_step_ratio', 'feedback', 'smooth_option', 'mp_physics', 'ra_lw_physics', 'ra_sw_physics', 'radt', 'sf_sfclay_physics', 'sf_surface_physics', 'bl_pbl_physics', 'bldt', 'cu_physics', 'cudt', 'hybrid_opt', 'rk_ord', 'diff_opt', 'km_opt', 'damp_opt', 'zdamp', 'dampcoef', 'khdif', 'kvdif', 'smdiv', 'emdiv', 'epssm', 'time_ste

In [39]:
########   STEP 3c: GET CONFIG DETAILS FROM CODE   #############
get_details_prompts={i:get_config_simple_2_with_config.format(variables='\n'.join(variables[i:i+10]),documentation=example_run_code_1_file_1) for i in range(0,len(variables),10)}
detailed_responses=process_ask_gpt_in_parallel(get_details_prompts.values(), get_details_prompts.keys(), model='gpt-4-1106-preview',max_workers=8,response_format={"type": "json_object"}) 
detailed_responses={key:json.loads(detailed_responses[key]) for key in detailed_responses.keys()}
variable_details={}
for key in detailed_responses.keys():
    for d in detailed_responses[key]['configuration_variable_details']:
        name=d['variable_name']
        d.pop('variable_name')
        variable_details[name]=d
variable_details

Processing Prompts: 100%|██████████| 9/9 [01:13<00:00,  8.20s/it]


{'io_form_history': {'type': 'int',
  'default': 2,
  'description': 'I/O format of history files',
  'options': 'Integers (For example: 1 for WRF format, 2 for another supported format)'},
 'io_form_restart': {'type': 'int',
  'default': 2,
  'description': 'I/O format of restart files',
  'options': 'Integers (For example: 1 for WRF format, 2 for another supported format)'},
 'io_form_input': {'type': 'int',
  'default': 2,
  'description': 'I/O format of input files',
  'options': 'Integers (For example: 1 for WRF format, 2 for another supported format)'},
 'io_form_boundary': {'type': 'int',
  'default': 2,
  'description': 'I/O format of boundary files',
  'options': 'Integers (For example: 1 for WRF format, 2 for another supported format)'},
 'time_step': {'type': 'int',
  'default': 600,
  'description': 'Time step size in seconds',
  'options': 'Positive integers, usually several factors of model resolution'},
 'time_step_fract_num': {'type': 'int',
  'default': 0,
  'descripti

In [40]:
print(len(variable_details))

86


In [49]:
print(example_run_code_1_file_1)

 &time_control
 run_days                            = 7,
 run_hours                           = 0,
 run_minutes                         = 0,
 run_seconds                         = 0,
 start_year                          = 1, 1, 1,
 start_month                         = 1,   1,   1,
 start_day                           = 1,   1,   1,
 start_hour                          = 0,   0,   0,
 start_minute                        = 0,   0,   0,
 start_second                        = 0,   0,   0,
 end_year                            = 1, 1, 1,
 end_month                           = 1, 1, 1,
 end_day                             = 5, 5, 5,
 end_hour                            = 0, 0, 0,
 end_minute                          = 0, 0, 0,
 end_second                          = 0, 0, 0,
 history_interval                    = 360, 360, 360,
 frames_per_outfile                  = 1000, 1000, 1000,
 restart                             = .false.,
 restart_interval                    = 3600,
 io_form_history 

In [50]:
########   STEP 4: CREATE CONFIG MODIFICATION FUNCTIONS FROM CODE  #############    
function_time=time.time()
functions=get_modification_functions(variables,example_run_code_1_file_1)
print('Took this much time to get functions to modify config - ',time.time()-function_time)

You are an AI that only responds with python code, NOT ENGLISH. You will be given a function signature and its docstring by the user. Write your full implementation (restate the function signature).
Use a Python code block to write your response. For example:
```python
print('Hello world!')
```
Write a function which changes the symmetric_ye,
open_ys,
open_ye,
nio_tasks_per_group,
nio_groups,
ideal_case values in the text below to user specified values. 
Note that the text below will ALWAYS be EXACTLY the same. 
You can take advantage of this fact in your function. 
Here is the function signature and doc string - 
def answer(original_text,
symmetric_ye,
open_ys,
open_ye,
nio_tasks_per_group,
nio_groups,
ideal_case):
#takes user supplied inputs and original text exactly in the form given and outputs text with the supplied variable values changed

Here is the original text -  
 &time_control
 run_days                            = 7,
 run_hours                           = 0,
 run_minutes 

In [13]:
#############   STEP 5: GET OUTPUT VARIABLES FROM OUTPUT FILES/DOCS  ####################

output_file_prompt=HOW_TO_RUN_PROMPT_TEMPLATE_part_4.format(library_name=library_name,article=example_run_doc_1) 
read_doc_time=time.time()
out = ask_gpt(output_file_prompt, model='gpt-4-1106-preview',response_format={"type": "json_object"})
print('Took this much time to get output files from docs - ',time.time()-read_doc_time)
output_file_extract=json.loads(out)
output_files=output_file_extract['output_file_names']
output_variables=output_file_extract['output_variable_names']

Took this much time to get output files from docs -  30.427591800689697


In [14]:
#############   STEP 6: GET RUN COMMANDS  ####################

run_commands_prompt=HOW_TO_RUN_PROMPT_TEMPLATE_part_3.format(library_name=library_name,article=example_run_doc_1) 
read_doc_time=time.time()
out = ask_gpt(run_commands_prompt, model='gpt-4-1106-preview',response_format={"type": "json_object"})
print('Took this much time to get run commands from docs - ',time.time()-read_doc_time)
run_commands_extract=json.loads(out)
run_commands=run_commands_extract['run_instructions']
run_commands

Took this much time to get run commands from docs -  29.65898895263672


['cd WRF/test/em_b_wave',
 'Edit the namelist.input file',
 './ideal.exe >& ideal.log or mpirun -np 1 ./ideal.exe',
 './wrf.exe >& wrf.log or mpirun -np 8 ./wrf.exe']

In [10]:
print(example_run_code_1_file_1)

 &time_control
 run_days                            = 7,
 run_hours                           = 0,
 run_minutes                         = 0,
 run_seconds                         = 0,
 start_year                          = 1, 1, 1,
 start_month                         = 1,   1,   1,
 start_day                           = 1,   1,   1,
 start_hour                          = 0,   0,   0,
 start_minute                        = 0,   0,   0,
 start_second                        = 0,   0,   0,
 end_year                            = 1, 1, 1,
 end_month                           = 1, 1, 1,
 end_day                             = 5, 5, 5,
 end_hour                            = 0, 0, 0,
 end_minute                          = 0, 0, 0,
 end_second                          = 0, 0, 0,
 history_interval                    = 360, 360, 360,
 frames_per_outfile                  = 1000, 1000, 1000,
 restart                             = .false.,
 restart_interval                    = 3600,
 io_form_history 

In [51]:
########### MODIFY CODE #################
new_config=use_generate_functions_to_modify_config(functions,variable_details,{'run_days':9,'dx':100000,'dy':100000},example_run_code_1_file_1)
with open(os.path.join('/media/hdd/Code/Build_WRF/WRF/test/em_b_wave/namelist.input'),'w') as f:
    f.write(new_config)

run_days
dx
dy


In [32]:
############### RUN CODE ###################
print(run_commands_extract['run_simulation_description'])
run_commands

To run the 3-D Baroclinic Wave simulation, first change the directory to WRF/test/em_b_wave. Then edit and save the namelist.input file for the simulation settings. For the initialization, execute ideal.exe with a single processor, and check 'ideal.log' or 'rsl.out.0000' for a success message. Finally, run the WRF model with './wrf.exe' for a serial build or 'mpirun -np 8 ./wrf.exe' for a parallel build using 8 processors, and verify successful completion by the appearance of the output and restart files, as well as a success message in 'wrf.log' or 'rsl.out.0000'.


['cd WRF/test/em_b_wave',
 'Edit the namelist.input file',
 './ideal.exe >& ideal.log or mpirun -np 1 ./ideal.exe',
 './wrf.exe >& wrf.log or mpirun -np 8 ./wrf.exe']